In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')

lable3 = pd.read_csv("lable3.csv")
#data_test = pd.read_csv("tap_fun_test.csv")

lable3.shape

(4549, 110)

In [29]:
# print lable3['prediction_pay_price'].value_counts()
print lable3.describe()

       wood_add_value  wood_reduce_value  stone_add_value  stone_reduce_value  \
count    4.549000e+03       4.549000e+03     4.549000e+03        4.549000e+03   
mean     3.809155e+06       3.480454e+06     1.327520e+06        1.073830e+06   
std      3.494822e+06       3.208604e+06     1.800170e+06        1.528234e+06   
min      0.000000e+00       0.000000e+00     0.000000e+00        0.000000e+00   
25%      8.875470e+05       6.707350e+05     2.000000e+04        0.000000e+00   
50%      3.051118e+06       2.781126e+06     6.100330e+05        3.304000e+05   
75%      5.872274e+06       5.483640e+06     2.087700e+06        1.753392e+06   
max      4.204610e+07       2.770304e+07     3.783471e+07        2.005761e+07   

       ivory_add_value  ivory_reduce_value  meat_add_value  meat_reduce_value  \
count     4.549000e+03        4.549000e+03    4.549000e+03       4.549000e+03   
mean      4.636513e+05        5.549035e+04    4.967758e+06       3.473389e+06   
std       6.860851e+05     

In [10]:
lable3['lable'] = 0
lable3['lable'] = lable3.apply(lambda x: 1 if(x['prediction_pay_price'] == 0.99 ) else x['lable'],axis=1)
print 'lable1'
lable3['lable'] = lable3.apply(lambda x: 2 if(x['prediction_pay_price'] > 0.99) else x['lable'],axis=1)
print 'lable2'

lable1
lable2


In [11]:
lable1 = lable3[lable3['lable']==1]
print lable1.shape
lable2 = lable3[lable3['lable']==2]
print lable2.shape

(1353, 110)
(3196, 110)


In [8]:
lable1.to_csv('lable1.csv',index=None)
lable2.to_csv('lable2.csv',index=None)
lable3.to_csv('lable3.csv',index=None)
lable4.to_csv('lable4.csv',index=None)

In [12]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score


In [13]:
lable3.drop(['user_id','prediction_pay_price','register_time','pay_price','pay_count'],axis=1, inplace=True)
lable3.shape

(4549, 105)

In [6]:
x, y = lable3.iloc[:, 0:len(lable3.columns)-1], lable3.iloc[:,len(lable3.columns)-1:]
x.shape,y.columns

((4549, 104), Index([u'lable'], dtype='object'))

In [7]:
# x = MinMaxScaler().fit_transform(x)
x = StandardScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.7)

In [15]:
lr = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), cv=3)
lr.fit(x_train, y_train)
print u'参数alpha=%.2f' % lr.alpha_
y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)
print u'Logistic回归训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'Logistic回归测试集准确率：', accuracy_score(y_test, y_test_pred)

rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, oob_score=True)
rf.fit(x_train, y_train)
print u'OOB Score=%.5f' % rf.oob_score_
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)
print u'随机森林训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'随机森林测试集准确率：', accuracy_score(y_test, y_test_pred)


参数alpha=1000.00
Logistic回归训练集准确率： 0.718475073313783
Logistic回归测试集准确率： 0.6938775510204082
OOB Score=0.71701
随机森林训练集准确率： 0.7595307917888563
随机森林测试集准确率： 0.6960753532182103


In [8]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, max_depth=6)
gb.fit(x_train, y_train)
y_train_pred = gb.predict(x_train)
y_test_pred = gb.predict(x_test)
print u'GBDT训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'GBDT测试集准确率：', accuracy_score(y_test, y_test_pred)

GBDT训练集准确率： 0.9904692082111437
GBDT测试集准确率： 0.6747252747252748


In [21]:

def lightGBM(train_x,train_y,validation_x,validation_y):
    clf = lgb.lgb.LGBMClassifier(boosting_type='gbdt',
                                 num_class=4, 
                                 num_leaves=55, 
                                 reg_alpha=0.0, 
                                 reg_lambda=1,
                                 max_depth=12, 
                                 n_estimators=1000, 
                                 objective='multiclass',
                                 subsample=0.8, 
                                 colsample_bytree=0.8, 
                                 subsample_freq=1,
                                 learning_rate=0.01, 
                                 min_child_weight=1, 
                                 random_state=20, 
                                 n_jobs=4
                                )
    
    # clf.fit(train_x, train_y, **fit_params)
    clf.fit(train_x,
    train_y,
    early_stopping_rounds=100,
    eval_metric= 'auc', 
            eval_set = [(validation_x,validation_y)],
            eval_names = ['valid'],
            verbose= 100
    )
         
    return clf

clf = lightGBM(x_train,y_train, x_test, y_test)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
print u'lgb训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'lgb测试集准确率：', accuracy_score(y_test, y_test_pred)
# param_grid = {
#     'learning_rate': [0.01, 0.1,0.2, 1],
#     'n_estimators': [100, 400]
# }
 
# gscv = GridSearchCV(estimator, param_grid, cv = 3, scoring="neg_mean_squared_error")
 
# gscv.fit(X_train, y_train)
 
# print('Best parameters found by grid search are:', gscv.best_params_)

Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.534281
Early stopping, best iteration is:
[45]	valid's auc: 0.544052
lgb训练集准确率： 0.7082111436950147
lgb测试集准确率： 0.7001569858712716


In [39]:
y_test_pred = pd.DataFrame(y_test_pred)
y_test = pd.DataFrame(y_test)

print y_test_pred.shape,y_test.shape
#
right3 = y_test_pred[(y_test_pred[0]==2)]
print right3.shape

(3185, 1) (3185, 1)
(3185, 1)
